# Projeto: Mini Mercado Inteligente

**Objetivo:** construir um notebook completo integrando ETL, análise descritiva, mineração de regras (Apriori), similaridade (Jaccard) e visualizações para uma rede de supermercados do Vale do Ribeira.

**Entrega:** `Projeto_MiniMercado.ipynb` — contém explicações, código e gráficos.

---

## 1) Importações e geração do dataset simulado

Nesta seção criamos um dataset de vendas simulado com clientes, produtos e transações.

In [ ]:

# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import datetime
import random
import os

random.seed(42)
np.random.seed(42)

# Parâmetros do dataset
n_customers = 200    # número de clientes únicos
n_products = 50      # número de produtos diferentes
n_transactions = 2000  # número de transações

# Gerar produtos
products = [f"Produto_{i:02d}" for i in range(1, n_products+1)]
prices = np.round(np.random.uniform(2.0, 120.0, size=n_products), 2)
product_df = pd.DataFrame({'product_id': products, 'price': prices})

# Gerar clientes
customers = [f"Cliente_{i:03d}" for i in range(1, n_customers+1)]

# Gerar transações: cada transação tem um cliente, data, e uma lista de produtos comprados
transactions = []
for t in range(n_transactions):
    customer = random.choice(customers)
    # número de itens por transação (mais prob. compras pequenas)
    size = np.random.choice([1,2,3,4,5,6], p=[0.25,0.25,0.2,0.15,0.1,0.05])
    items = list(np.random.choice(products, size=size, replace=False))
    # assign quantities and compute transaction total
    quantities = list(np.random.randint(1,4,size=len(items)))
    date = datetime.date(2025, random.randint(1,12), random.randint(1,28))
    transactions.append({'transaction_id': f"T{t+1:05d}", 'customer_id': customer, 'date': pd.to_datetime(date), 'items': items, 'quantities': quantities})

# transformar em DataFrame 'explodido' (cada linha = um item da transação)
rows = []
for tr in transactions:
    for prod, qty in zip(tr['items'], tr['quantities']):
        price = float(product_df.loc[product_df['product_id']==prod,'price'].values[0])
        rows.append({'transaction_id': tr['transaction_id'], 'customer_id': tr['customer_id'], 'date': tr['date'], 'product_id': prod, 'quantity': qty, 'price': price, 'total': qty*price})

sales = pd.DataFrame(rows)
sales.reset_index(drop=True, inplace=True)
print('Dataset criado: linhas =', len(sales))
sales.head()

## 2) ETL: carregar e tratar a base

Etapas realizadas:
- verificação de tipos
- remoção de possíveis duplicatas
- agregação por transação quando necessário


In [ ]:

# ETL simples e verificações
df = sales.copy()
# tipos
df['date'] = pd.to_datetime(df['date'])
# remover duplicatas estranhas
df = df.drop_duplicates()

# criar coluna 'transaction_total' agregada por transaction_id
transaction_totals = df.groupby('transaction_id')['total'].sum().reset_index().rename(columns={'total':'transaction_total'})
df = df.merge(transaction_totals, on='transaction_id', how='left')

# quick checks
print('Transações únicas:', df['transaction_id'].nunique())
print('Clientes únicos:', df['customer_id'].nunique())
print('Produtos únicos nas vendas:', df['product_id'].nunique())

df.head()

## 3) Análise Descritiva

Gerar estatísticas sobre produtos, valores e quantidades. Mostraremos: top produtos por quantidade e por receita, e estatísticas resumo.

In [ ]:

# Top 5 produtos por quantidade vendida
top_q = df.groupby('product_id')['quantity'].sum().sort_values(ascending=False).head(10)
top_q5 = top_q.head(5)

# Top 5 produtos por receita
top_rev = df.groupby('product_id')['total'].sum().sort_values(ascending=False).head(10)
top_rev5 = top_rev.head(5)

# Estatísticas gerais
stats = {
    'total_transactions': df['transaction_id'].nunique(),
    'total_customers': df['customer_id'].nunique(),
    'total_revenue': df['total'].sum(),
    'avg_items_per_transaction': df.groupby('transaction_id')['product_id'].nunique().mean(),
    'avg_transaction_value': df.groupby('transaction_id')['transaction_total'].first().mean()
}
print(stats)
display(top_q5, top_rev5)


### Visualização: Top 5 produtos mais vendidos (quantidade e receita)

In [ ]:

# Plot Top 5 quantidade and receita
fig, axes = plt.subplots(1,2, figsize=(12,5))
top_q5.sort_values().plot.barh(ax=axes[0], title='Top 5 por Quantidade (unidades)')
top_rev5.sort_values().plot.barh(ax=axes[1], title='Top 5 por Receita (R$)')
plt.tight_layout()
plt.show()


## 4) Preparar dados para Apriori

Criar o formato 'basket' (cada transação = conjunto de produtos) necessário para mineração de regras de associação.

In [ ]:

# Agrupar produtos por transação
basket = df.groupby('transaction_id')['product_id'].apply(lambda x: list(set(x))).reset_index()
basket.head()

## 5) Mineração (Apriori)

Implementamos uma versão simples do algoritmo Apriori para encontrar itemsets frequentes e gerar regras com suporte, confiança e lift.

In [ ]:

from collections import defaultdict
from itertools import combinations

def find_frequent_itemsets(transactions, min_support=0.01):
    N = len(transactions)
    item_counts = defaultdict(int)
    for t in transactions:
        for item in t:
            item_counts[frozenset([item])] += 1
    freq_itemsets = {}
    L1 = {itemset: count/N for itemset, count in item_counts.items() if count/N >= min_support}
    current_L = L1
    k = 2
    freq_itemsets.update(current_L)
    while current_L:
        candidates = set()
        prev_itemsets = list(current_L.keys())
        for a,b in combinations(prev_itemsets,2):
            union = a.union(b)
            if len(union) == k:
                candidates.add(union)
        candidate_counts = defaultdict(int)
        for t in transactions:
            tset = set(t)
            for c in candidates:
                if c.issubset(tset):
                    candidate_counts[c] += 1
        current_L = {c: count/N for c,count in candidate_counts.items() if count/N >= min_support}
        freq_itemsets.update(current_L)
        k += 1
    return freq_itemsets

def generate_rules(freq_itemsets, transactions, min_confidence=0.2):
    supports = freq_itemsets
    rules = []
    for itemset in [iset for iset in supports.keys() if len(iset)>=2]:
        for r in range(1, len(itemset)):
            for antecedent in combinations(itemset, r):
                antecedent = frozenset(antecedent)
                consequent = itemset - antecedent
                if antecedent in supports and itemset in supports:
                    conf = supports[itemset] / supports[antecedent]
                    lift = supports[itemset] / (supports[antecedent]*supports[consequent]) if (supports.get(consequent,0)>0) else np.nan
                    if conf >= min_confidence:
                        rules.append({'antecedent': antecedent, 'consequent': consequent, 'support': supports[itemset], 'confidence': conf, 'lift': lift})
    rules = sorted(rules, key=lambda x: x['lift'] if not np.isnan(x['lift']) else 0, reverse=True)
    return rules

transactions_list = basket['product_id'].tolist()
freq_itemsets = find_frequent_itemsets(transactions_list, min_support=0.02)
len(freq_itemsets)


In [ ]:

rules = generate_rules(freq_itemsets, transactions_list, min_confidence=0.25)
# show top 10 rules
rules[:10]


### Regras de Associação mais fortes (exibir top por lift)

In [ ]:

# Mostrar as principais regras em DataFrame
rules_df = pd.DataFrame([{'antecedent': ','.join(sorted(list(r['antecedent']))),
                          'consequent': ','.join(sorted(list(r['consequent']))),
                          'support': r['support'],
                          'confidence': r['confidence'],
                          'lift': r['lift']} for r in rules])
rules_df = rules_df.sort_values('lift', ascending=False).reset_index(drop=True)
rules_df.head(10)


## 6) Similaridade (Jaccard) entre clientes

Construímos uma matriz binária cliente x produto e calculamos a similaridade Jaccard para identificar clientes com padrões semelhantes.

In [ ]:

# Criar matriz cliente x produto (binary: 1 se cliente já comprou o produto)
cust_prod = df.groupby(['customer_id','product_id']).size().unstack(fill_value=0)
cust_prod = (cust_prod > 0).astype(int)

# compute pairwise Jaccard similarity
from sklearn.metrics import jaccard_score
customers = cust_prod.index.tolist()
X = cust_prod.values
n = X.shape[0]
jaccard_mat = np.zeros((n,n))
for i in range(n):
    for j in range(i,n):
        if i==j:
            jaccard_mat[i,j] = 1.0
        else:
            score = jaccard_score(X[i], X[j])
            jaccard_mat[i,j] = score
            jaccard_mat[j,i] = score

jaccard_df = pd.DataFrame(jaccard_mat, index=customers, columns=customers)
jaccard_df.iloc[:6,:6]


### Visualização: Mapa de similaridade entre clientes

Exibimos um heatmap reduzido e uma rede com as ligações de maior similaridade.

In [ ]:

# Heatmap (plot subset of customers to keep figure readable)
subset = customers[:40]
plt.figure(figsize=(10,8))
plt.imshow(jaccard_df.loc[subset, subset].values, aspect='auto', interpolation='nearest')
plt.colorbar(label='Jaccard similarity')
plt.title('Heatmap de Similaridade Jaccard (subset 40 clientes)')
plt.xlabel('Clientes')
plt.ylabel('Clientes')
plt.xticks(ticks=np.arange(len(subset)), labels=[s.replace('Cliente_','') for s in subset], rotation=90, fontsize=6)
plt.yticks(ticks=np.arange(len(subset)), labels=[s.replace('Cliente_','') for s in subset], fontsize=6)
plt.tight_layout()
plt.show()


In [ ]:

# Rede: conectar pares com Jaccard alto (>0.6) - somente top pares para visualização
import networkx as nx
G = nx.Graph()
threshold = 0.6
for i in range(n):
    for j in range(i+1, n):
        if jaccard_mat[i,j] >= threshold:
            G.add_edge(customers[i], customers[j], weight=jaccard_mat[i,j])

plt.figure(figsize=(10,7))
if len(G.nodes)>0:
    pos = nx.spring_layout(G, seed=42)
    nx.draw_networkx_nodes(G, pos, node_size=50)
    nx.draw_networkx_edges(G, pos, alpha=0.6)
    nx.draw_networkx_labels(G, pos, font_size=6)
    plt.title(f'Rede de clientes com Jaccard >= {threshold} (n_edges={len(G.edges)})')
    plt.axis('off')
    plt.show()
else:
    print('Nenhuma aresta com Jaccard >=', threshold, ' — experimente diminuir o threshold.')


## 7) Resumo e próximos passos

- Notebook inclui ETL, análise descritiva, Apriori e similaridade Jaccard.
- Próximos passos sugeridos: ajustar thresholds de Apriori, incluir métricas temporais, integrar com dashboard interativo (Streamlit) e testes A/B para promoções.

---

Arquivo pronto para download abaixo.